我们以一个基本的静态网站作为案例进行爬取，需要爬取的链接为： https://static1.scrape.cuiqingcai.com/   
本节的目标：  
1.用 requests 爬取这个站点每一页的电影列表，顺着列表再爬取每个电影的详情页。  
2.用 pyquery 和正则表达式提取每部电影的名称、封面、类别、上映时间、评分、剧情简介等。  
3.把以上爬取的内容存入 MongoDB 数据库。  
4.使用多进程实现爬取的加速。  

In [1]:
import requests #  用来爬取页面
import logging #  用来输出信息
import re # 用来实现正则表达式解析
import pymongo  # 用来实现 MongoDB  存储
from pyquery import PyQuery as pq # 用来直接解析网页
from urllib.parse import urljoin # 用来做 URL 的拼接
from requests.packages import urllib3
urllib3.disable_warnings() # 忽略证书警告

1. 定义日志输出级别和输出格式

In [2]:
logging.basicConfig(level=logging.INFO, # 输出级别
                   format='%(asctime)s-%(levelname)s:%(message)s') # 输出格式

2. 定义根URL

In [3]:
BASE_URL = 'https://static1.scrape.cuiqingcai.com'

3. 爬取总页数

In [4]:
TOTAL_PAGE = 10

4. 定义函数：接收一个url参数，返回页面的html.text代码。  

In [5]:
def scrape_page(url):
    logging.info(f'正在爬取{url}……')
    try:
        response = requests.get(url, verify=False)
    except requests.RequestException:
        logging.error('爬取%s时发生了一个错误', url, exc_info=True)
        # 这时，将logging.error的exc_info参数设置为True可以打印出Traceback错误信息。
    else:
        if response.status_code == 200:
            return response.text
        else:
            logging.error(f'爬取{url}失败，错误代码{response.status_code}。')

5. 定义函数：接收页码参数, 返回该页的html.text代码

In [6]:
def scrape_index(p):
    index_url = f'{BASE_URL}/page/{p}'
    return scrape_page(index_url)

6. 定义函数：接收html.text代码, 返回该页每部电影网址（拼接后）

In [7]:
def parse_index(html):
    doc = pq(html)
    links = doc('.el-card .name')
    for i in links.items():
        href = i.attr('href') # 找到小网址
        detail_url = urljoin(BASE_URL, href) # 拼接起来，扔到地址生成器中
        yield detail_url # 返回一个地址生成器

7. 利用上面定义的函数 逐页提取所有电影的url

In [8]:
movie_url = []
for page in range(1, TOTAL_PAGE+1):
    html = scrape_index(page)
    detail_url = parse_index(html)
    for i in detail_url:
        movie_url.append(i)
movie_url

2021-10-11 15:13:14,488-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/1……
2021-10-11 15:13:14,756-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/2……
2021-10-11 15:13:15,024-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/3……
2021-10-11 15:13:15,273-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/4……
2021-10-11 15:13:15,556-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/5……
2021-10-11 15:13:15,831-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/6……
2021-10-11 15:13:16,108-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/7……
2021-10-11 15:13:16,371-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/8……
2021-10-11 15:13:16,638-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/9……
2021-10-11 15:13:16,894-INFO:正在爬取https://static1.scrape.cuiqingcai.com/page/10……


['https://static1.scrape.cuiqingcai.com/detail/1',
 'https://static1.scrape.cuiqingcai.com/detail/2',
 'https://static1.scrape.cuiqingcai.com/detail/3',
 'https://static1.scrape.cuiqingcai.com/detail/4',
 'https://static1.scrape.cuiqingcai.com/detail/5',
 'https://static1.scrape.cuiqingcai.com/detail/6',
 'https://static1.scrape.cuiqingcai.com/detail/7',
 'https://static1.scrape.cuiqingcai.com/detail/8',
 'https://static1.scrape.cuiqingcai.com/detail/9',
 'https://static1.scrape.cuiqingcai.com/detail/10',
 'https://static1.scrape.cuiqingcai.com/detail/11',
 'https://static1.scrape.cuiqingcai.com/detail/12',
 'https://static1.scrape.cuiqingcai.com/detail/13',
 'https://static1.scrape.cuiqingcai.com/detail/14',
 'https://static1.scrape.cuiqingcai.com/detail/15',
 'https://static1.scrape.cuiqingcai.com/detail/16',
 'https://static1.scrape.cuiqingcai.com/detail/17',
 'https://static1.scrape.cuiqingcai.com/detail/18',
 'https://static1.scrape.cuiqingcai.com/detail/19',
 'https://static1.scr

8. 定义函数：复用scrape_page，做成一个新的scrape_detail方法。

必要性说明：  
你可能会问，这个scrape_detail方法里面只调用了scrape_page方法，没有别的功能，那爬取详情页直接用scrape_page方法不就好了，还有必要再单独定义scrape_detail方法吗？    
答案是有必要，单独定义一个scrape_detail方法在逻辑上会显得更清晰，而且以后如果我们想要对scrape_detail方法进行改动，比如添加日志输出或是增加预处理，都可以在scrape_detail里面实现，而不用改动scrape_page方法，灵活性会更好。

In [9]:
def scrape_detail(url):
    return scrape_page(url)

9. 定义函数：接收html.text代码, 返回电影详情页爬取结果的字典

In [11]:
def parse_detail(html):
    doc = pq(html)
    cover = doc('img.cover').attr('src')
    name = doc('a > h2').text()
    categories = [item.text() for item in doc('.categories button span').items()]
    published_at = doc('.info:contains( 上映 )').text()
    published_at = re.search('(\d{4}-\d{2}-\d{2})', published_at).group(1) \
        if published_at and re.search('\d{4}-\d{2}-\d{2}', published_at) else None
    drama = doc('.drama p').text()
    score = doc('p.score').text()
    score = float(score) if score else None
    return {
        'cover': cover,
        'name': name,
        'categories': categories,
        'published_at': published_at,
        'drama': drama,
        'score': score
    }



a = scrape_detail('https://static1.scrape.cuiqingcai.com/detail/1')
a = parse_detail(a)
a

2021-10-11 15:13:26,033-INFO:正在爬取https://static1.scrape.cuiqingcai.com/detail/1……


{'cover': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c',
 'name': '霸王别姬 - Farewell My Concubine',
 'categories': ['剧情', '爱情'],
 'published_at': '1993-07-26',
 'drama': '影片借一出《霸王别姬》的京戏，牵扯出三个人之间一段随时代风云变幻的爱恨情仇。段小楼（张丰毅 饰）与程蝶衣（张国荣 饰）是一对打小一起长大的师兄弟，两人一个演生，一个饰旦，一向配合天衣无缝，尤其一出《霸王别姬》，更是誉满京城，为此，两人约定合演一辈子《霸王别姬》。但两人对戏剧与人生关系的理解有本质不同，段小楼深知戏非人生，程蝶衣则是人戏不分。段小楼在认为该成家立业之时迎娶了名妓菊仙（巩俐 饰），致使程蝶衣认定菊仙是可耻的第三者，使段小楼做了叛徒，自此，三人围绕一出《霸王别姬》生出的爱恨情仇战开始随着时代风云的变迁不断升级，终酿成悲剧。',
 'score': 9.5}

10. MongoDB数据库准备

In [12]:
client = pymongo.MongoClient('mongodb://localhost:27017/') # 连接MongoDB
db = client['movies'] # 指定数据库
collection = db['movies'] # 指定集合

11. 定义函数：将数据保存到MongoDB的方法。

In [13]:
def save_data(data):
    collection.update_one(
        {'name': data.get('name')}, 
        {'$set': data}, 
        upsert=True # 存在即更新，不存在即插入
    )

12. 抓取+保存

In [ ]:
for i in movie_url:
    html = scrape_detail(i)
    data = parse_detail(html)
    logging.info('爬取到数据：%s', data)
    logging.info('保存MongoDB中')
    save_data(data)
    logging.info('数据保存成功')

### 多进程加速
由于整个的爬取是单进程的，而且只能逐条爬取，速度稍微有点慢，有没有方法来对整个爬取过程进行加速呢？  
在前面我们讲了多进程的基本原理和使用方法，下面我们就来实践一下多进程的爬取吧。  
由于一共有 10  页详情页，并且这 10  页内容是互不干扰的，所以我们可以一页开一个进程来爬取。由于这 10  个列表页页码正好可以提前构造成一个列表，所以我们可以选用多进程里面的进程池 Pool  来实现这个过程。

In [ ]:
import multiprocessing

def main(page):
    index_html = scrape_index(page)
    detail_urls = parse_index(index_html)
    for detail_url in detail_urls:
        detail_html = scrape_detail(detail_url)
        data = parse_detail(detail_html)
        logging.info('get detail data %s', data)
        logging.info('saving data to mongodb')
        save_data(data)
        logging.info('data saved successfully')

if __name__ == '__main__':
    pool = multiprocessing.Pool()
    pages = range(1, TOTAL_PAGE + 1)
    pool.map(main, pages)
    pool.close()
    pool.join()